# OPTIMIZING MODEL PARAMETERS

- Training a model is an iterative process that involves several steps.
- In each iteration, the model makes predictions, calculates the error (loss) between its predictions and the actual targets.
- The derivatives of the error with respect to the model's parameters are computed through backpropagation.
- Gradient descent is then used to optimize these parameters based on the computed gradients.
- The model repeats this process for multiple iterations, gradually improving its performance by minimizing the error and updating the parameters accordingly.

## Prerequisite Code


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

Hyperparameters are adjustable parameters that allow us to control the model optimization process. These values can significantly influence the training and convergence rates of the model.

For training, we define the following hyperparameters:

1. Number of Epochs: This determines how many times the model iterates over the entire dataset during training.

2. Batch Size: It indicates the number of data samples that are propagated through the network before updating the model's parameters. 

3. Learning Rate: This parameter controls the magnitude of updates to the model's parameters at each batch or epoch. Smaller values result in slower learning, while larger values can lead to unpredictable behavior during training.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

##  Optimization Loop


After defining the hyperparameters, we can proceed with training and optimizing our model using an optimization loop. In this process, each iteration of the optimization loop is referred to as an epoch.

An epoch comprises two primary components:
1. Train Loop: Iterate over the training dataset and optimize model parameters.

2. Validation/Test Loop: Iterate over the test dataset to evaluate model performance and monitor improvement.

## Loss Function


- Untrained networks often produce incorrect predictions on training data.
- The loss function quantifies the dissimilarity between predictions and target values, which we aim to minimize during training.
- Common loss functions are nn.MSELoss for regression tasks and nn.NLLLoss for classification tasks.
- For multi-class classification, nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss.

**We pass our model’s output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.**

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

- Optimization aims to adjust model parameters to minimize model error during training.
- Optimization algorithms dictate the specific approach for parameter adjustment.
- The optimizer object encapsulates all the logic related to the optimization process.
- In the example, Stochastic Gradient Descent (SGD) is used as the optimization algorithm.

**We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.**

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:

Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

Backpropagate the prediction loss with a call to loss.backward(). PyTorch deposits the gradients of the loss w.r.t. each parameter.

Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302296  [   64/60000]
loss: 2.295867  [ 6464/60000]
loss: 2.273243  [12864/60000]
loss: 2.268357  [19264/60000]
loss: 2.259188  [25664/60000]
loss: 2.223239  [32064/60000]
loss: 2.237178  [38464/60000]
loss: 2.198077  [44864/60000]
loss: 2.194753  [51264/60000]
loss: 2.167207  [57664/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 2.165053 

Epoch 2
-------------------------------
loss: 2.166389  [   64/60000]
loss: 2.164572  [ 6464/60000]
loss: 2.105277  [12864/60000]
loss: 2.124672  [19264/60000]
loss: 2.072789  [25664/60000]
loss: 2.012524  [32064/60000]
loss: 2.046674  [38464/60000]
loss: 1.964018  [44864/60000]
loss: 1.964232  [51264/60000]
loss: 1.895697  [57664/60000]
Test Error: 
 Accuracy: 60.1%, Avg loss: 1.898666 

Epoch 3
-------------------------------
loss: 1.922370  [   64/60000]
loss: 1.900261  [ 6464/60000]
loss: 1.785633  [12864/60000]
loss: 1.824746  [19264/60000]
loss: 1.704462  [25664/60000]
loss: 1.657972  [32064/600